# The stability of the global average on different window lengths
This notebook outllines different methods that can be used to de-trend the LongRunMIP and ZECMIP time series.

# Imports

In [1]:
import nc_time_axis
import warnings

import xarray as xr
import numpy as np
import pandas as pd
from importlib import reload
from numba import njit
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.gridspec as gridspec 
from matplotlib.axes import Axes
import matplotlib as mpl
import time

import os
import sys
import string

from numpy.typing import ArrayLike
from typing import Optional, Callable, List, Tuple, Dict

warnings.filterwarnings('ignore')

In [2]:
sys.path.append(os.path.join(os.getcwd(), 'Documents/PhD'))

In [3]:
import constants
from constants import PlotConfig
sys.path.append(constants.MODULE_DIR)
import sn_plotting
import xarray_class_accessors as xca
import utils
import stats
import xarray_extender as xce
import signal_to_noise as sn
import open_ds
from classes import ExperimentTypes, LocationsLatLon
import plotting_functions
import xarray_extender as xe

logger = utils.get_notebook_logger()

In [4]:
notebook_number='01'

In [5]:
%matplotlib inline

In [6]:
# import dask.distributed as dd
# import tempfile

# tempdir = tempfile.TemporaryDirectory("dask-worker-space")
# dd.Client(local_directory=tempdir.name, memory_limit='20gb')

# Loading Data

In [7]:
experiment_params = constants.EXPERIMENTS_TO_RUN[0]
experiment_params

{'variable': 'tas', 'mask': None, 'hemisphere': 'global'}

In [8]:
constants.ZECMIP_LOCAL_REGRIDDED_DIR,

('/g/data/w40/ab2313/PhD/zecmip/regridded',)

In [9]:
experiment_zec_da = open_ds.open_mfdataset_nc(os.path.join(constants.ZECMIP_LOCAL_REGRIDDED_DIR, 'A1'), 
                                              to_array=False).tas
picontrol_zec_da = open_ds.open_mfdataset_nc(os.path.join(constants.ZECMIP_LOCAL_REGRIDDED_DIR, 'picontrol'),
                                             to_array=False).tas

In [10]:
experiment_zec_ds_smean, picontrol_zec_ds_smean = sn.calculate_global_value(
    experiment_zec_da, picontrol_zec_da, experiment_params["variable"])

(None, None)


In [11]:
SN_DIR = os.path.join(constants.ZECMIP_LOCAL_REGRIDDED_DIR, 'signal_to_noise')
SN_DIR

'/g/data/w40/ab2313/PhD/zecmip/regridded/signal_to_noise'

In [12]:
local_signal_to_noise_files = [f for f in  os.listdir(SN_DIR) if 'A1' not in f]
local_signal_to_noise_files

['exp_rolling_noise_False.nc',
 'control_rolling_noise_False.nc',
 'control_rolling_noise_True.nc',
 'exp_rolling_noise_True.nc']

In [13]:
# Opening local data into dictionary
local_data_obj = {}
for fname in local_signal_to_noise_files:
    ds = xr.open_dataset(os.path.join(SN_DIR, fname)).chunk('auto')
    # LOL. OMFGGGGGGGGGG!!!! Don't need all of these windows.
    local_data_obj[fname] = ds.sel(window=[21, 41]).persist()

print(list(local_data_obj.keys()))

['exp_rolling_noise_False.nc', 'control_rolling_noise_False.nc', 'control_rolling_noise_True.nc', 'exp_rolling_noise_True.nc']


# Local

### Calculations

In [14]:
def calculate_and_add_upper_and_lower_bound(ds:xr.Dataset, control_ds:xr.Dataset)->xr.Dataset:
    '''
    Takes the picontrol from a run and also the experiment in questions. The upper and lower bound 
    are calculated and added to the dataset
    '''
    control__lbound, control__ubound = sn.calculate_upper_and_lower_bounds(control_ds.signal_to_noise,
                                                                           logginglevel='INFO')
    
    ds_w_bounds = xe.add_lower_upper_to_dataset(ds.signal_to_noise,
                                                control__lbound, control__ubound)
    return ds_w_bounds

In [ ]:
exp_local_sn_static_ds = calculate_and_add_upper_and_lower_bound(
    local_data_obj['exp_rolling_noise_False.nc'],
    local_data_obj['control_rolling_noise_False.nc'].isel(time=slice(1,100)).chunk({'time':-1, 'lat':-1})
    ).persist()

- Calculating Upper and lower control bounds
- Map blocks used


In [15]:
exp_local_sn_rolling_ds = calculate_and_add_upper_and_lower_bound(
    local_data_obj['exp_rolling_noise_True.nc'],
    local_data_obj['control_rolling_noise_True.nc'].isel(time=slice(1,100)).chunk({'time':-1, 'lat':-1})
    ).persist()

- Calculating Upper and lower control bounds
- Map blocks used
ERROR! Session/line number was not unique in database. History logging moved to new session 1095



KeyboardInterrupt



In [ ]:
# Fraction of models stable at each time.
# static_fraction_stable_local = sn.get_fraction_stable_ds(exp_local_sn_ds)
static_year_stable_local = sn.get_stable_year_ds(exp_local_sn_static_ds).persist()
rolling_year_stable_local = sn.get_stable_year_ds(exp_local_sn_rolling_ds).persist()

In [ ]:
static_year_stable_local_average = xe.get_average_and_uncertainty_across_dim(
    static_year_stable_local, 'model', averaging_method='mean', uncertainty_method='max_min')

rollling_year_stable_local_average = xe.get_average_and_uncertainty_across_dim(
    rolling_year_stable_local, 'model', averaging_method='mean', uncertainty_method='max_min')

static_year_stable_local_average

### Plotting

In [ ]:
bound = 3
step=0.25
signal_to_noise_levels = np.arange(-bound, bound+step, step)

In [ ]:
max_effective_lenght = 100

In [ ]:

fig = plt.figure(figsize=(22,16))
gs = gridspec.GridSpec(3, 2, height_ratios=[1, 1, 0.1])
ax1 = fig.add_subplot(gs[0, 0], projection=ccrs.PlateCarree())
ax2 = fig.add_subplot(gs[0, 1], projection=ccrs.PlateCarree())
ax3 = fig.add_subplot(gs[1, 0], projection=ccrs.PlateCarree())
ax4 = fig.add_subplot(gs[1, 1], projection=ccrs.PlateCarree())

plot_kwargs = dict(levels=signal_to_noise_levels, cmap='RdBu_r', add_colorbar=False, extend='both')

window=21
exp_local_sn_static_ds.lower_bound.mean(dim='model').sel(window=window).plot(ax=ax1, **plot_kwargs)
cplot = exp_local_sn_static_ds.upper_bound.mean(dim='model').sel(window=window).plot(ax=ax2, **plot_kwargs)

window=41
exp_local_sn_static_ds.lower_bound.mean(dim='model').sel(window=window).plot(ax=ax3, **plot_kwargs)
exp_local_sn_static_ds.upper_bound.mean(dim='model').sel(window=window).plot(ax=ax4, **plot_kwargs)

sn_plotting.format_plot(fig, ax1)
sn_plotting.format_plot(fig, ax2)
sn_plotting.format_plot(fig, ax3)
sn_plotting.format_plot(fig, ax4)

ax1.set_title('Lower Bound', fontsize=16)
ax2.set_title('Upper Bound', fontsize=16)
fig.suptitle(f'Multi-model Mean Bounds ({window=})', fontsize=22, y=0.9)


cax = plt.subplot(gs[2, :])
cbar = plt.colorbar(cplot, cax=cax, orientation='horizontal')
cbar.ax.set_title('Signal-to-noise')

In [ ]:

fig = plt.figure(figsize=(22,16))
gs = gridspec.GridSpec(3, 2, height_ratios=[1, 1, 0.1])
ax1 = fig.add_subplot(gs[0, 0], projection=ccrs.PlateCarree())
ax2 = fig.add_subplot(gs[0, 1], projection=ccrs.PlateCarree())
ax3 = fig.add_subplot(gs[1, 0], projection=ccrs.PlateCarree())
ax4 = fig.add_subplot(gs[1, 1], projection=ccrs.PlateCarree())

plot_kwargs = dict(levels=signal_to_noise_levels, cmap='RdBu_r', add_colorbar=False, extend='both')

window=21
exp_local_sn_rolling_ds.lower_bound.mean(dim='model').sel(window=window).plot(ax=ax1, **plot_kwargs)
cplot = exp_local_sn_rolling_ds.upper_bound.mean(dim='model').sel(window=window).plot(ax=ax2, **plot_kwargs)

window=41
exp_local_sn_rolling_ds.lower_bound.mean(dim='model').sel(window=window).plot(ax=ax3, **plot_kwargs)
exp_local_sn_rolling_ds.upper_bound.mean(dim='model').sel(window=window).plot(ax=ax4, **plot_kwargs)

sn_plotting.format_plot(fig, ax1)
sn_plotting.format_plot(fig, ax2)
sn_plotting.format_plot(fig, ax3)
sn_plotting.format_plot(fig, ax4)

ax1.set_title('Lower Bound', fontsize=16)
ax2.set_title('Upper Bound', fontsize=16)
fig.suptitle(f'Multi-model Mean Bounds ({window=})', fontsize=22, y=0.9)


cax = plt.subplot(gs[2, :])
cbar = plt.colorbar(cplot, cax=cax, orientation='horizontal')
cbar.ax.set_title('Signal-to-noise')

In [ ]:
((rollling_year_stable_local_average.squeeze().drop('variable').to_array(name='new') -
    static_year_stable_local_average.squeeze().drop('variable').to_array(name='new')).
 isel(window=0).sel(variable='average_value')).plot()

In [ ]:
fig, axes, plots = sn_plotting.map_plot_all_for_coords(
    rollling_year_stable_local_average[['average_value']] -\
    static_year_stable_local_average[['average_value']], hspace=.6, 
    column_coord='window', column_title_tag='Years', y=1.2, row_labels=['Mean'],
    variable=experiment_params['variable'], extend='neither', cmap='RdBu')


save_name = os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL,
                         f'{notebook_number}_global_average_year_of_diff.png')
print(save_name)
fig.savefig(save_name, dpi=300, bbox_inches='tight')

In [ ]:
reload(sn_plotting)
reload(plotting_functions)

In [ ]:
fig, axes, plots = sn_plotting.map_plot_all_for_coords(
    rollling_year_stable_local_average.squeeze().drop('variable').to_array(name='new'), column_coord='window',
    column_title_tag='Years', y=1.05, row_coord='variable', row_labels=['Mean', 'Uncertainty'],
    variable=experiment_params['variable'], extend='max', cmap='Blues', vmin=0, one_colorbar=True)

save_name = os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL,
                         f'{notebook_number}_local_average_year_of_stabilisation_rolling.png')
print(save_name)
fig.savefig(save_name, dpi=300, bbox_inches='tight')

In [ ]:
fig, axes, plots = sn_plotting.map_plot_all_for_coords(
    static_year_stable_local_average.squeeze().drop('variable').to_array(name='new'), column_coord='window',
    column_title_tag='Years', y=1.05, row_coord='variable', row_labels=['Mean', 'Uncertainty'],
    variable=experiment_params['variable'], extend='max', cmap='Blues', vmin=0, one_colorbar=True)

save_name = os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL,
                         f'{notebook_number}_local_average_year_of_stabilisation_static.png')
print(save_name)
fig.savefig(save_name, dpi=300, bbox_inches='tight')

In [ ]:
fig, axes, plots = sn_plotting.map_plot_all_for_coords(
    static_year_stable_local, column_coord='window', row_coord='model', y=0.92,
    variable=experiment_params['variable'], extend='max', cmap='Blues', vmin=0)

save_name = os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL,
                         f'{notebook_number}_all_model_year_of_stabilisation_static.png')
print(save_name)
fig.savefig(save_name, dpi=300, bbox_inches='tight')

In [ ]:
fig, axes, plots = sn_plotting.map_plot_all_for_coords(
    rolling_year_stable_local, column_coord='window', row_coord='model', y=0.92,
    variable=experiment_params['variable'], extend='max', cmap='Blues', vmin=0)

save_name = os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL,
                         f'{notebook_number}_all_model_year_of_stabilisation_rolling.png')
print(save_name)
fig.savefig(save_name, dpi=300, bbox_inches='tight')

# Global

## Calculations

In [ ]:
trend_da = stats.trend_fit(experiment_zec_ds_smean, method='polynomial', order=3)
detrended_da = experiment_zec_ds_smean - trend_da

In [ ]:
sn_multi_static_ds = sn.multiwindow_signal_to_nosie_and_bounds(
    experiment_zec_ds_smean, picontrol_zec_ds_smean,
    rolling_noise=False, da_for_noise = detrended_da, parallel=False,
    logginglevel='ERROR', return_all=True,
    **constants.ZECMIP_MULTI_WINDOW_RUN_PARAMS)

In [ ]:
sn_multi_rolling_ds = sn.multiwindow_signal_to_nosie_and_bounds(
    experiment_zec_ds_smean, picontrol_zec_ds_smean,
    rolling_noise=True, da_for_noise = detrended_da, parallel=False,
    logginglevel='ERROR', return_all=True,
    **constants.ZECMIP_MULTI_WINDOW_RUN_PARAMS)

In [ ]:
max_effective_length = 50

In [ ]:
# Number of models stable at each time step.
rolling_stability_levels_ds = sn.stability_levels(sn_multi_rolling_ds)
static_stability_levels_ds = sn.stability_levels(sn_multi_static_ds)

# Year when models first become stable.
rolling_year_stable = sn.get_stable_year_ds(sn_multi_rolling_ds, max_effective_length=max_effective_length)
static_year_stable = sn.get_stable_year_ds(sn_multi_static_ds, max_effective_length=max_effective_length)

In [ ]:
unstable_rolling = sn_multi_rolling_ds.utils.above_or_below(
                                'signal_to_noise', greater_than_var='upper_bound', less_than_var='lower_bound')
unstable_rolling

In [ ]:
@njit()
def mean_after_index(arr, index, window):
    val_array = []
    for num,ind in enumerate(index):
        vals = arr[num, ind:ind+window]
        val_array.append(np.nanmean(vals))
    return val_array

In [ ]:
def index_over_dim(da:xr.DataArray, index_da:xr.DataArray)->xr.DataArray:
    arr = da.values; val_stor = []
    for window in index_da.window.values:
        index = index_da.sel(window=window).squeeze().time.values
        val_array = mean_after_index(arr, index, window=window)
        val_stor.append(np.array(val_array))
    return np.array(val_stor)

In [ ]:
stable_temp_ds = xr.zeros_like(rolling_year_stable.squeeze().rename({'time':'temp'})).to_array().astype(float)

stable_temp_ds += index_over_dim(experiment_zec_ds_smean.squeeze().transpose(), rolling_year_stable)

In [ ]:
experiment_zec_da.shape

In [ ]:
rolling_year_stable_local.isel(window=0).squeeze().time.shape

In [ ]:
experiment_zec_da

In [ ]:
experiment_zec_da_anom = experiment_zec_da - experiment_zec_da.isel(time=slice(None, 10)).mean(dim='time')

In [ ]:
experiment_zec_da_anom.isel(model=0)

In [ ]:
arr1 = experiment_zec_da.values
arr2 = rolling_year_stable_local.isel(window=0).squeeze().time.values

In [ ]:
I, J, K = np.ix_(np.arange(8), np.arange(96), np.arange(192))

In [ ]:
anomaly_at_index_ds = xr.zeros_like(rolling_year_stable_local.isel(window=0).squeeze().rename({'time':'temp'}))
anomaly_at_index_ds += arr1[arr2.astype(int), I, J, K]

In [ ]:
anomaly_at_index_ds.temp.isel(model=0).plot()

# Plotting

In [ ]:
model = 2

In [ ]:
# Manual Calculation for Demonstations
WINDOW=25
# Sample period is the years that we want to look at
sample_period = (0,WINDOW) #(1200,1261), 61

# The length of the sample_period varible
window_length = WINDOW

# Subset
sample_data = experiment_zec_ds_smean.isel(time=slice(*sample_period), model=model)
sample_values = sample_data.values
x = np.arange(len(sample_values))

# Gradient and y-intercept of the sample data (use=[0,1] will return grad and y intercept)
p = np.polyfit(x=x, y=sample_values, deg=1)
fitted_line = np.polyval(p, x=x)

In [ ]:
plt.style.use('seaborn-darkgrid')
fig = plt.figure(figsize=(12,18))
axes = [fig.add_subplot(3,1,i) for i in range(1,3)]


# ---> axes[0]
sample_data.plot(ax=axes[0], marker='o', label='Simulation', color='green')
axes[0].plot(sample_data.time.values, fitted_line, label='Lienar Trend Fit', color='blue')
axes[0].set_xlim(sample_data.time.values[0], sample_data.time.values[-1])
axes[0].legend(ncol=2, fontsize=constants.PlotConfig.legend_text_size)

plotting_functions.format_axis(axes[0], title='Signal Calculation', xlabel='Time (years)', ylabel='GMST\n'+r'Anomaly ($^{\circ}C$)', labelpad=55)
plotting_functions.add_figure_label(axes[0], 'a)')

# ---> axes[1]
ax1 = axes[1]; ax2 = axes[1].twinx()
lns1 = experiment_zec_ds_smean.isel(model=model).plot(ax=ax1, color='green', label='Simulation')
lns2 = trend_da.isel(model=model).plot(ax=ax1, label='Trend Line', color='green', linestyle=':')
lns3 = detrended_da.isel(model=model).plot(ax=ax2, label='De-trended Simulation', color='blue')

ax1.set_yticks([0.8, 1, 1.2, 1.4, 1.6, 1.8])
ax1.set_ylim([0.8, 1.8])
ax2_ytick_locations=np.linspace(ax2.get_yticks()[0], ax2.get_yticks()[-1], len(ax1.get_yticks()))
ax2.set_yticks(ax2_ytick_locations)

lns = lns1+lns2+lns3
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, ncol=3, loc='best', fontsize=constants.PlotConfig.legend_text_size)

plotting_functions.format_axis(ax1, title='Noise Calculation', xlabel='Time (years)', ylabel='GMST\n'+r'Anomaly ($^{\circ}C$)', labelpad=55)
plotting_functions.format_axis(ax2, title='Noise Calculation', ylabel='GMST\nAnomaly\nDetrended' + r'($^{\circ}C$)', labelpad=55)
plotting_functions.add_figure_label(ax1, 'b)')

fig.savefig(os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL, f'{notebook_number}_method_demonstration.png'), bbox_inches='tight', dpi=300)

In [ ]:
models = experiment_zec_ds_smean.model.values
ncols = 2
nrows = int(np.ceil(len(models)/ncols))

fig = plt.figure(figsize=(8*ncols, 3*nrows))
for plot_num, model in enumerate(models):
    ax = fig.add_subplot(nrows,ncols,plot_num+1)
    experiment_zec_ds_smean.sel(model=model).plot(ax=ax, color='red')
    trend_da.sel(model=model).plot(ax=ax, linewidth=2.5, color='green')
    detrended_da.sel(model=model).plot(ax=ax, color='blue')

In [ ]:
fig, ax = sn_plotting.plot_median_stable_year(static_year_stable, rolling_year_stable)

fig.savefig(os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL,
                         f'{notebook_number}_stabilisation_year_double_poly_vs_static.png'),
           dpi=300)

In [ ]:
fig, ax = sn_plotting.plot_stable_year_all_models(static_year_stable-rolling_year_stable, exp_type='zecmip')

save_name = os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL,
                         f'{notebook_number}_global_average_year_of_stabilisation_diff.png')
print(save_name)
fig.savefig(save_name, dpi=300, bbox_inches='tight')

In [ ]:
fig, ax = sn_plotting.plot_stable_year_all_models(static_year_stable, exp_type='zecmip')
save_name = os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL,
                         f'{notebook_number}_global_average_year_of_stabilisation_static.png')
print(save_name)
fig.savefig(save_name, dpi=300, bbox_inches='tight')

In [ ]:
fig, ax = sn_plotting.plot_stable_year_all_models(rolling_year_stable, exp_type='zecmip')
save_name = os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL, f'{notebook_number}_global_average_year_of_stabilisation_rolling.png')
print(save_name)
fig.savefig(save_name, dpi=300, bbox_inches='tight')

In [ ]:
fig, ax = sn_plotting.plot_stable_year_all_models(stable_temp_ds_2.to_dataset(name='time'), exp_type='zecmip', 
                                                 xlabel=f'Global Mean\n{longname} Anomaly\n({units})')

In [ ]:
reload(sn_plotting)
reload(plotting_functions)

In [ ]:
fig = plt.figure(figsize=(35, 16))
gs = gridspec.GridSpec(1,2, wspace=0.25)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
reload(sn_plotting)
fig, ax = sn_plotting.plot_stable_year_all_models(rolling_year_stable, exp_type='zecmip', ncol=5, 
                                                 fig=fig, ax=ax1, legend_loc = 'top_ofset',
                                                 bbox_to_anchor=(2.1, 1.15), font_scale=1.5)

fig, ax =sn_plotting.plot_stable_year_all_models(stable_temp_ds_2.to_dataset(name='time'), fig=fig, ax=ax2, exp_type='zecmip', 
                                                 xlabel=f'Global Mean\n{longname} Anomaly\n({units})',
                                                 add_legend=False, font_scale=1.5, xlabelpad=60)

save_name = os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL,
                         f'{notebook_number}_global_average_year_of_stabilisation_and_temp_anomaly.png')
print(save_name)
fig.savefig(save_name, dpi=300, bbox_inches='tight')

In [ ]:
reload(sn_plotting)

In [ ]:
fig = plt.figure(figsize=(35, 16))
gs = gridspec.GridSpec(1,2, wspace=0.25)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
reload(sn_plotting)
fig, ax = sn_plotting.plot_stable_year_all_models(rolling_year_stable, exp_type='zecmip', ncol=5, 
                                                 fig=fig, ax=ax1, legend_loc = 'top_ofset',
                                                 bbox_to_anchor=(2.1, 1.15), font_scale=1.5)
fig, ax = sn_plotting.plot_stable_year_all_models(static_year_stable, exp_type='zecmip', 
                                                 fig=fig, ax=ax2, add_legend=False, font_scale=1.5)
save_name = os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL,
                         f'{notebook_number}_global_average_year_of_stabilisation_static_and_rolling.png')
print(save_name)
fig.savefig(save_name, dpi=300, bbox_inches='tight')

In [ ]:
units = constants.VARIABLE_INFO[experiment_params["variable"]]["units"]
longname = constants.VARIABLE_INFO[experiment_params["variable"]]['longname']
hemisphere_title = string.capwords(experiment_params['hemisphere'].replace('_', ' '))
mask_title = f'({experiment_params["mask"]} only)' if experiment_params["mask"] else ''

In [ ]:
plt.style.use('seaborn-darkgrid')
fig, ax = sn_plotting.plot_all_coord_lines(picontrol_zec_ds_smean, exp_type='zecmip', xlabel='Time (Years)',
                              ylabel=f'Global Mean\n{longname}\n({units})', ncol=1)
save_name = os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL, f'{notebook_number}_global_average_picontrol_time_series.png'); print(save_name)
# fig.savefig(save_name, dpi=300, bbox_inches='tight')

In [ ]:
plt.style.use('seaborn-darkgrid')
fig, ax = sn_plotting.plot_all_coord_lines(experiment_zec_ds_smean, exp_type='zecmip',
                               xlabel='Time (Years)', ylabel=f'Global Mean\n{longname}\nAnomaly({units})', ncol=1)
save_name = os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL,
                         f'{notebook_number}_global_average_A1_time_series.png')
print(save_name)
fig.savefig(save_name, dpi=300, bbox_inches='tight')

In [ ]:
models = rolling_year_stable.model.values
levels = np.arange(0, len(models)+1,1)
my_blues = plotting_functions.create_discrete_cmap('Blues', levels=levels, add_white=True)
my_reds = plotting_functions.create_discrete_cmap('Reds', levels=levels, add_white=True)

plot_kwargs = dict(exp_type='zecmip',vmin=0, vmax=len(models)+1, step=1, tick_offset='center',
                   extend='neither', cbar_label='Number of Unstable Models')

comparison_kwargs = dict(max_color_lim=50, xlims=(0,50), font_scale=1.6)

In [ ]:
mpl.rcParams.update(mpl.rcParamsDefault)

fig = plt.figure(figsize=(sn_plotting.plot_kwargs['width'],
                          sn_plotting.plot_kwargs['height']*2.1))

gs = gridspec.GridSpec(3,1, height_ratios=[1, 1, 0.1], hspace=sn_plotting.plot_kwargs['hspace'])
axes = [fig.add_subplot(gs[i]) for i in range(3)]

sn_plotting.sn_multi_window_in_time(
    static_stability_levels_ds.sel(stability='unstable').count(dim='model'), cmap=my_blues,
    fig=fig, gs=gs, ax=axes[0], add_colorbar=False, return_all=False, axes_title='Static Noise',
    **plot_kwargs, **comparison_kwargs)

sn_plotting.sn_multi_window_in_time(
    rolling_stability_levels_ds.sel(stability='unstable').count(dim='model'), cmap=my_blues,
    fig=fig, gs=gs, ax=axes[1], cax=axes[2], return_all=False, axes_title='Rolling Noise',
    **plot_kwargs, **comparison_kwargs)


save_name = os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL,
   f'{notebook_number}_global_average_stabilisation_pattern_comparison_static_and_rolling.png')
print(save_name)
fig.savefig(save_name, dpi=300, bbox_inches='tight')

In [ ]:
reload(sn_plotting)
%matplotlib inline

In [ ]:
stability_colorplot_kwargs = dict(max_color_lim=50, xlims=(0, 90),patch=True,
                                  ax2_ylabel=f'Global Mean\n{longname}\nAnomaly({units})')

In [ ]:
models = unstalbe_rolling.model.values
sn_levels = np.arange(-4, 4+.5, 0.5)

fig = plt.figure(figsize=(sn_plotting.plot_kwargs['width'], sn_plotting.plot_kwargs['height']*len(models)/1.5))
gs = gridspec.GridSpec(len(models)+1, 1, height_ratios=len(models)*[1] + [0.2], hspace=sn_plotting.plot_kwargs['hspace']*1.1)
axes = [fig.add_subplot(gs[i]) for i in range(len(models))]

for num, model in enumerate(models):
    add_colorbar=False; cax=None
    if num == len(models)-1: add_colorbar=True; cax=fig.add_subplot(gs[num+1])
    sn_plotting.sn_multi_window_in_time(
        unstable_rolling.sel(model=model).signal_to_noise, cmap='RdBu_r',
        fig=fig, gs=gs, ax=axes[num], cax=cax, add_colorbar=add_colorbar,
        axes_title=model, bbox_to_anchor=(1, 1.4),
        temp_da=experiment_zec_ds_smean.sel(model=model),
        stable_point_ds = rolling_year_stable.sel(model=model), labelpad=80,
        extend='both', cut_ticks=2, vmax=4, step=0.5,  font_scale=1.4, cbar_label='Signal-To-Noise Ratio',
        hspace=0.2, return_all=False, **stability_colorplot_kwargs)  
    
for num, ax in enumerate(axes): plotting_functions.add_figure_label(ax, f'{chr(97+num)})', font_scale=1.4)
    
save_name = os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL,
                         f'{notebook_number}_global_all_model_sn_pattern.png')
print(save_name)
fig.savefig(save_name, dpi=300, bbox_inches='tight')

In [ ]:
%matplotlib inline

In [ ]:
reload(plotting_functions)
reload(sn_plotting)

In [ ]:
fig = plt.figure(figsize=(sn_plotting.plot_kwargs['width'], sn_plotting.plot_kwargs['height']*2.1))

gs = gridspec.GridSpec(3,1, height_ratios=[1, 1, 0.1], hspace=sn_plotting.plot_kwargs['hspace'])
axes = [fig.add_subplot(gs[i]) for i in range(3)]

sn_plotting.sn_multi_window_in_time(
    rolling_stability_levels_ds.sel(stability='increasing').count(dim='model'), cmap=my_blues,
    fig=fig, gs=gs, ax=axes[0], add_colorbar=False, return_all=False,
    axes_title='Increasing Temperature Trends', bbox_to_anchor=(1, 1.4),
    temp_da=experiment_zec_ds_smean,  **stability_colorplot_kwargs,  **plot_kwargs)

sn_plotting.sn_multi_window_in_time(
    rolling_stability_levels_ds.sel(stability='decreasing').count(dim='model'), cmap=my_blues,
    fig=fig, gs=gs, ax=axes[1], cax=axes[2], return_all=False,
    axes_title='Decreasing Temperature Trends',
    add_legend=False,
    temp_da=experiment_zec_ds_smean, **stability_colorplot_kwargs,  **plot_kwargs)


for num, ax_ in enumerate(axes):
    plotting_functions.add_figure_label(ax_, f'{chr(97+num)})')

save_name = os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL,
                         f'{notebook_number}_global_average_stabilisation_pattern_rolling.png')
print(save_name)
fig.savefig(save_name, dpi=300, bbox_inches='tight')

In [ ]:
%matplotlib inline

In [ ]:
fig, axes = sn_plotting.sn_multi_window_in_time(
    static_stability_levels_ds.sel(stability='increasing').count(dim='model'),
    temp_da=experiment_zec_ds_smean, cmap=my_reds, **stability_colorplot_kwargs,  **plot_kwargs)
fig

save_name = os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL,
                         f'{notebook_number}_global_average_stabilisation_pattern_warming_static.png')
print(save_name)
fig.savefig(save_name, dpi=300, bbox_inches='tight')
fig

In [ ]:
fig, axes = sn_plotting.sn_multi_window_in_time(
    static_stability_levels_ds.sel(stability='decreasing').count(dim='model'),
    temp_da=experiment_zec_ds_smean, cmap=my_blues, **stability_colorplot_kwargs,  **plot_kwargs)

save_name = os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL,
                         f'{notebook_number}_global_average_stabilisation_pattern_cooling_static.png')
print(save_name)
fig.savefig(save_name, dpi=300, bbox_inches='tight')
fig

In [ ]:
noise_levels = sn_plotting.create_levels(vmax=.13, vmin=0.1, step=.002)

In [ ]:
%matplotlib inline

In [ ]:
sn_plotting.plot_heatmap(sn_multi_rolling_ds.noise.mean(dim='model'), levels=noise_levels, 
                                figsize=(sn_plotting.plot_kwargs['width']/3,
                                         sn_plotting.plot_kwargs['height']/2), 
                               cmap='RdBu', extend='both', cut_ticks=2, max_color_lim=50, 
                               font_scale=0.8, cbar_label='Noise', hspace=0.2, return_all=False)

save_name = os.path.join(constants.IMAGE_SAVE_DIR_TOP_LEVEL,
                         f'{notebook_number}_global_average_noise_rolling.png')
print(save_name)
fig.savefig(save_name, dpi=300, bbox_inches='tight')

In [ ]:
percent_frac = sn_multi_rolling_ds.noise*100/sn_multi_static_ds.noise
diff = sn_multi_rolling_ds.noise - sn_multi_static_ds.noise
percent_diff = diff * 100/sn_multi_rolling_ds.noise

In [ ]:
# plot_kwargs_2 = dict(height=12, width=22, hspace=0.3, #vmin=-8, vmax=8, step=2, 
#                    cmap = 'RdBu_r', line_color = 'limegreen', line_alpha=0.65, 
#                    ax2_ylabel = 'Anomaly', cbar_label = 'Signal-to-Noise', cbartick_offset=0,
#                    axes_title='',
#                    title='', label_size=12, extend='both', xlowerlim=None, xupperlim=None,  filter_max=True,)